# **Challenge 1**
## Bridging Medical Deserts - Building Intelligent Document Parsing Agents for the Virtue Foundation
### Sponsored Track by DataBricks


## **_Cempasuchil Team_**


In this notebook you will find all the processes behind the building of our multi-agent structure.
Even if we used only create_react_agent call, the way it functions is multi-agent in nature: it acts as a Supervisor that autonomously delegates tasks to specialized Tools ("Specialized Agents" or "Skill Agents").

In [0]:
# This cell installs the core dependencies for Databricks Vector Search and LangChain.
%pip install langchain databricks-vectorsearch

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# This cell was used to upload and restart the Python interpreter within the cluster each time we uploaded versions
dbutils.library.restartPython()

**Phase 2**

In this phase we taught the agent how to audit one hospital.

This cell implements the core logic for identifying "Infrastructure Gaps."  It uses a "Verify-with-Retry" pattern to ensure high-fidelity extraction from unstructured field reports. 

**Key Technical Features:**
1. Structured Extraction: Uses Llama 3.3 to convert free-form auditor notes into machine-readable JSON.
2. Dynamic Ground-Truth Matching: Cross-references extracted data against the official Unity Catalog records (Delta Table/CSV).
3. Anomaly Detection: Calculates discrepancies between 'Official Capacity' and 'Observed Reality,' specifically targeting suspicious claims or resource inflation as required by the Virtue Foundation challenge.

**_Note_**

Why Llama 3.3?
--------------------------------------------------------------------------------
We selected Meta's Llama 3.3-70B-Instruct served via Databricks Model Serving 
over other proprietary LLMs for three critical reasons: 

1. DATA SOVEREIGNTY: Running Llama within the Databricks ecosystem ensures  sensitive medical infrastructure data never leaves our secure environment.
2. STATE-OF-THE-ART REASONING: Llama 3.3 70B provides performance comparable to much larger models (like GPT-4), making it ideal for the complex logic required to detect "Medical Deserts" and suspicious claims.
3. PERFORMANCE & COST: Using a model optimized for Databricks Model Serving allows for lower latency and higher throughput, enabling the "100x speed increase" in auditing requested by the challenge.

In [0]:
import time
import json
import pandas as pd
from openai import OpenAI
from databricks.sdk import WorkspaceClient

# 1. INITIALIZE
w = WorkspaceClient()
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
host = ctx.apiUrl().getOrElse(None)
token = ctx.apiToken().getOrElse(None)
client = OpenAI(api_key=token, base_url=f"{host}/serving-endpoints")

MODEL = "databricks-meta-llama-3-3-70b-instruct"

def extract_with_retry(text, retries=3):
    """Retries the LLM call if it returns an empty string."""
    prompt = f"Extract to JSON: facility_name, beds_observed (int), has_oxygen (bool). Text: {text}"
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}
            )
            content = response.choices[0].message.content
            if content and len(content.strip()) > 0:
                return json.loads(content)
        except Exception as e:
            print(f"Attempt {i+1} failed: {e}")
            time.sleep(2) # Wait 2 seconds before retrying
    return None

# RUN AUDIT
csv_path = "/Volumes/workspace/default/raw_medical_reports/Virtue Foundation Ghana v0.3 - Sheet1.csv"
df = pd.read_csv(csv_path)

mock_text = """
FIELD AUDIT REPORT: Aneeja Hospital
Location: Tantra Hill, Accra, Ghana
Assessment Date: Feb 2026

Observations:
The audit team performed a floor-by-floor inspection of Aneeja Hospital. 
We confirmed a total of 40 functional inpatient beds (including 3 pediatric VIP wards). 
The facility is well-equipped with oxygen supply systems and two active operating theatres.
Staffing levels are consistent with a 24/7 emergency service.
"""
ground_truth = extract_with_retry(mock_text)

if ground_truth:
    # Match using 'name' column from your CSV
    match = df[df['name'].str.contains(ground_truth['facility_name'], case=False, na=False)]
    if not match.empty:
        official = match.iloc[0]
        # Audit logic using 'capacity' column
        off_cap = pd.to_numeric(official['capacity'], errors='coerce') or 0
        obs_cap = ground_truth['beds_observed']
        
        print("\n--- AUDIT SUCCESS ---")
        print(f"Facility: {official['name']}")
        print(f"Discrepancy: Official {int(off_cap)} vs Observed {obs_cap}")
    else:
        print("Facility match not found in CSV.")
else:
    print("CRITICAL: LLM failed to respond after retries.")


--- AUDIT SUCCESS ---
Facility: Aneeja Hospital
Discrepancy: Official 39 vs Observed 40


**Phase 3**

Here we are giving the agent a library by establishing a "Single Source of Truth" in Unity Catalog.
We performed Data Engineering to prepare the foundation for our Agentic Intelligence. 

**Key Technical Steps:**
1. We clean and rename columns to be  SQL-compatible (removing spaces and special characters), ensuring compatibility with Spark and Delta Lake.
2. DELTA LAKE CONVERSION: Persists the raw CSV data as a managed Delta Table in the 'workspace.default' catalog. This enables ACID transactions, schema evolution, and high-performance querying for the Auditor Agent.
3. GOVERNANCE: By centralizing the data in Unity Catalog, we ensure that the Agent is grounded in a governed, official data source, preventing hallucinations and ensuring row-level accountability.

In [0]:
import pandas as pd

# Load the CSV from Volume
csv_path = "/Volumes/workspace/default/raw_medical_reports/Virtue Foundation Ghana v0.3 - Sheet1.csv"
df = pd.read_csv(csv_path)

# Clean column names 
# Spark/Delta tables do not allow spaces, dots, or slashes in column names
df.columns = [c.replace(' ', '_').replace('.', '_').replace('/', '_').replace('(', '').replace(')', '').lower() for c in df.columns]

# Save as a Delta Table in 'workspace' catalog
target_table = "workspace.default.ghana_health_facilities"

try:
    # Convert Pandas to Spark and Save
    spark.createDataFrame(df).write.mode("overwrite").option("mergeSchema", "true").saveAsTable(target_table)
    print(f" Success! Table created at: {target_table}")
    
    # Show the first few rows to confirm
    display(spark.table(target_table).limit(5))
    
except Exception as e:
    print(f" Error saving table: {e}")

 Success! Table created at: workspace.default.ghana_health_facilities


source_url,name,pk_unique_id,mongo_db,specialties,procedure,equipment,capability,organization_type,content_table_id,phone_numbers,email,websites,officialwebsite,yearestablished,acceptsvolunteers,facebooklink,twitterlink,linkedinlink,instagramlink,logo,address_line1,address_line2,address_line3,address_city,address_stateorregion,address_ziporpostcode,address_country,address_countrycode,countries,missionstatement,missionstatementlink,organizationdescription,facilitytypeid,operatortypeid,affiliationtypeids,description,area,numberdoctors,capacity,unique_id
https://www.linkedin.com/company/waaf/,"109/No 1 Bekwai Rd (Near Mexico Hotel) Takoradi, Ghana",1,62aa51490990af00169ab9ed,"[""infectiousDiseases"",""maternalFetalMedicineOrPerinatology"",""publicHealth"",""socialAndBehavioralSciences"",""hospiceAndPalliativeInternalMedicine"",""globalHealthAndInternationalHealth""]",null,null,"[""Has a location at 109/No 1 Bekwai Rd (Near Mexico Hotel) Takoradi, Ghana"",""Specialties include HIV, AIDS, PMTCT, Behavior Change Communication, HIV Testing and Counseling, Community Outreach, Stigma Reduction, Hospice / Home Based Care, global health, and public health""]",facility,a77400f4-6203-4b0d-84ad-fd143bd768e3,"[""+233249354576"",""+233203928883""]",null,"[""waafweb.org""]",waafweb.org,null,null,null,null,null,null,null,109/No 1 Bekwai Rd (Near Mexico Hotel),null,null,Takoradi,null,null,Ghana,GH,null,null,null,null,clinic,null,null,"WAAF is committed to battling HIV/AIDS, TB, and other conditions through grassroot initiatives throughout communities. The main objectives for WAAF are to deliver prevention, behavioral, care and support services for the general population but with emphasis on vulnerable populations. Our ultimate goal is a future where HIV and AIDS is no longer an epidemic, and people are no longer stigmatized. The organization’s vision and mission guide for daily and long-term operations, as detailed in the organization’s strategic plan document.",null,null,null,7d362eaa-8130-410a-bf23-7be549177af0
https://www.ghanabusinessweb.com/accra-adabraka-health_clinics-iran_clinic-9430.html,1st Foundation Clinic,2,null,"[""internalMedicine""]",[],[],"[""Located in Dansoman, Accra, Ghana, opposite Standard Chartered Bank"",""Listed as a related place on GhanaBusinessWeb's Iran Clinic page""]",facility,cd191c26-2987-404f-b5bb-7dfe6d7a7b02,null,null,null,null,null,null,null,null,null,null,null,Opp. Standard Chartered Bank,null,null,Dansoman,null,null,Ghana,GH,null,null,null,null,clinic,null,null,null,null,null,null,9d70e24f-247c-41af-b708-cc10e99e54b1
https://www.ghanabusinessweb.com/accra-cantonments-str_0-kumoji_hospital-9464.html,1st Foundation Clinic,2,null,"[""internalMedicine""]",[],[],[],facility,6cc7060e-63f3-4e20-b83c-483ac1c3206e,null,null,null,null,null,null,null,null,null,null,null,"Opp. Standard Chartered Bank, Dansoman",null,null,Accra,null,null,Ghana,GH,null,null,null,null,clinic,null,null,null,null,null,null,97d11408-8303-44c3-83c2-72cb91c58fb1
https://www.ghanabusinessweb.com/accra-dansoman-hospitals-karikari_brobbey_hospital-9444.html,1st Foundation Clinic,2,null,"[""internalMedicine""]",[],[],[],facility,2d84a935-452c-441c-a766-8fcbb8b4e7ea,null,null,null,null,null,null,null,null,null,null,null,Opp. Standard Chartered Bank,Dansoman,null,Accra,null,null,Ghana,GH,null,null,null,null,clinic,null,null,null,null,null,null,4c17951e-87e0-4472-8cf4-189cea9782b8
https://www.ghanabusinessweb.com/accra-osu-health_clinics-gak_clinic-9409.html,1st Foundation Clinic,2,null,"[""internalMedicine""]",null,null,null,facility,78038e2e-3210-4a0b-a502-ad0f8aabbb38,null,null,null,null,null,null,null,null,null,null,null,Opp. Standard Chartered Bank,Dansoman,null,Accra,null,null,Ghana,GH,null,null,null,null,clinic,null,null,null,null,null,null,a6ec226d-a88e-4366-b390-5c709ef54e92


**Connecting the Auditor to Vector Search**

Our aim was to overcome Data Ambiguity through Vector Search. In order to achieve it we initialized the Vector Search Client and defined the "Semantic Matcher." We did it by connecting the agent to the 'medical_desert_endpoint,' allowing the system to query high-dimensional embeddings of hospital descriptions.

Furthermore, this function understands medical context. It can match a snippet like "maternity 
focus in Accra" to the correct facility record even if the exact words don't appear in the database columns.

In [0]:
from databricks.vector_search.client import VectorSearchClient
import json

# Initialize the Vector Search Client
vsc = VectorSearchClient()

# Eendpoint name created in the UI ("medical_desert_endpoint")
VECTOR_ENDPOINT = "medical_desert_endpoint" 
INDEX_NAME = "workspace.default.ghana_health_index"

index = vsc.get_index(endpoint_name=VECTOR_ENDPOINT, index_name=INDEX_NAME)

def semantic_audit_search(report_snippet):
    """semantic_audit_search
    Searches the Vector Index for the most relevant facility 
    based on the text found in a field report.
    """
    print(f"Searching for: {report_snippet}...")
    
    results = index.similarity_search(
        query_text=report_snippet,
        columns=["name", "capacity", "description", "pk_unique_id"],
        num_results=1 # We want the single best match
    )
    
    # Check if results exist
    if results.get('result', {}).get('data_array'):
        match = results['result']['data_array'][0]
        # Mapping results: [name, capacity, description, pk_unique_id]
        return {
            "name": match[0],
            "capacity": match[1],
            "description": match[2],
            "id": match[3]
        }
    else:
        return None

# SEMANTIC AUDIT TEST
# Notice we don't use the exact name here—the agent will 'find' the match.
field_report_observation = "Small facility in Accra with approximately 40 beds and maternity focus."

best_match = semantic_audit_search(field_report_observation)

if best_match:
    print("\n--- SEMANTIC MATCH FOUND ---")
    print(f"Match: {best_match['name']}")
    print(f"Official Capacity: {best_match['capacity']}")

else:
    print("No matching facility found in the knowledge base.")

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
Searching for: Small facility in Accra with approximately 40 beds and maternity focus....
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.

--- SEMANTIC MATCH FOUND ---
Match: Mwin Tuba Hospital and Colo-Proctology Centre
Official Capacity: None


**Phase 4: Agents**

In [0]:
# Upgrade MLflow to version 3.0+, which is required for Databricks Agentic Tracing
%pip install mlflow>=3.0 --upgrade

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


This next steps construct the "Medical Integrity Agent," an autonomous reasoning engine built using the LangGraph ReAct pattern. 

**Key Architectural Components:**
1. SPECIALIZED SKILL TOOLS:
    - Structured Querying: Fetches precise official data from Unity Catalog.
    - Semantic Search: Resolves facility entities via Vectorized Indexes.    
    - IDP Extraction: Parses messy field reports into structured metrics.
    - Geospatial Math: Uses Haversine calculations to map "Medical Deserts."

 2. PROMPT: Implements strict input protocols for the LLM to prevent SQL hallucinations and ensure 100% auditable results.

 3. PLAN VISIBILITY: By streaming the agent's output, we satisfy the requirement for a "Transparent Planning System," making the AI's internal logic visible and trustworthy.

In [0]:
import json
import math
import mlflow
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from databricks.vector_search.client import VectorSearchClient

# Enable Tracing 
mlflow.langchain.autolog() 

# Initialize Brain
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
host = ctx.apiUrl().getOrElse(None)
token = ctx.apiToken().getOrElse(None)

llm = ChatOpenAI(
    openai_api_key=token,
    openai_api_base=f"{host}/serving-endpoints",
    model="databricks-meta-llama-3-3-70b-instruct",
    temperature=0
)

# TOOLS 

@tool
def query_hospital_database(sql_condition: str):
    """
    TOOL: OFFICIAL DATA LOOKUP.
    Input MUST be a valid SQL WHERE clause condition only. 
    Example: "equipment LIKE '%neurosurgery%'" or "capacity > 20".
    DO NOT provide full SELECT statements.
    """
    # We hardcode the SELECT part so the agent can't mess it up
    table = "workspace.default.ghana_health_facilities"
    full_sql = f"SELECT pk_unique_id, name, capacity, equipment, specialties FROM {table} WHERE {sql_condition}"
    
    try:
        df = spark.sql(full_sql).toPandas()
        if df.empty:
            return "No facilities found matching those criteria."
        return df.to_json(orient="records")
    except Exception as e:
        return f"SQL Syntax Error. Please provide a simpler WHERE condition. Error: {str(e)}"

@tool
def get_facility_official_data(facility_name_keyword: str):
    """
    Useful for finding official capacity, equipment, and IDs for a facility.
    Input should be a simple name like "Aneeja" or "Korle Bu".
    """
    # We write the SQL for the agent so it can't make syntax errors
    clean_name = facility_name_keyword.replace("'", "").strip()
    sql = f"""
    SELECT pk_unique_id, name, capacity, equipment, source_url 
    FROM workspace.default.ghana_health_facilities 
    WHERE name LIKE '%{clean_name}%'
    LIMIT 3
    """
    try:
        df = spark.sql(sql).toPandas()
        if df.empty:
            return "No facility found with that name."
        return df.to_json(orient="records")
    except Exception as e:
        return f"Database Error: {str(e)}"

@tool
def extract_medical_features(raw_text: str):
    """
    CORE FEATURE: UNSTRUCTURED FEATURE EXTRACTION (IDP).
    Parses messy field notes into structured data using an internal LLM call.
    """
    # This tool fulfills the Intelligent Document Parsing requirement
    prompt = f"Extract facility_name, beds_count (int), and oxygen_available (bool) from: {raw_text}"
    response = llm.invoke(prompt) # Using the LLM to parse the text
    return response.content

@tool
def search_medical_notes(description: str):
    """
    STRETCH GOAL: SEMANTIC CITATIONS.
    Uses Vector Search to find facilities based on meaning.
    """
    vsc = VectorSearchClient()
    index = vsc.get_index(endpoint_name="medical_desert_endpoint", index_name="workspace.default.ghana_health_index")

    # We return the pk_unique_id so the agent can cite its source
    results = index.similarity_search(
        query_text=description,
        columns=["pk_unique_id", "name", "capacity", "description"],
        num_results=2
    )
    return json.dumps(results['result']['data_array'])

@tool
def check_audit_discrepancy(official_beds: int, observed_beds: int):
    """
    Math tool to verify claims.
    Returns 'FLAGGED' if observed beds are much lower than official claims.
    """
    if official_beds > (observed_beds * 1.5): # Threshold: 50% inflation
        return f"FLAGGED: Significant Discrepancy. Official: {official_beds}, Observed: {observed_beds}."
    return "VERIFIED: Data matches within reasonable limits."

@tool
def calculate_distance_km(lat1: float, lon1: float, lat2: float, lon2: float):
    """
    STRETCH GOAL: GEOSPATIAL ANALYSIS.
    Calculates the real distance between two medical facilities using the Haversine formula.
    Use this to identify 'cold spots' or facilities within a specific radius.
    """
    try:
        # Haversine formula
        R = 6371.0  # Earth radius in km
        phi1, phi2 = math.radians(lat1), math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        
        a = math.sin(dphi / 2)**2 + \
            math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        
        return round(R * c, 2)
    except Exception as e:
        return f"Calculation Error: {str(e)}"


system_prompt = """
You are the Virtue Foundation Healthcare Integrity Agent. Your mission is to provide 100x faster healthcare auditing for Ghana by identifying "Medical Deserts" and verifying infrastructure claims.

--- TOOL PROTOCOLS ---
1. query_hospital_database: 
   - STRICT RULE: Your input must ONLY be the SQL WHERE condition. 
   - NEVER write "SELECT", "FROM", or "WHERE".
   - Correct Input Example: "equipment LIKE '%cardiology%' AND capacity > 10"
   - Incorrect Input Example: "SELECT * FROM table WHERE..."

2. extract_medical_features: Use this for any messy, unstructured text or field notes provided by the user.

3. calculate_distance_km: Use this for any geospatial questions involving "near", "how far", or "radius".

--- MUST-HAVE REASONING LOGIC ---
- For "How many..." questions: Use query_hospital_database to get the list, then count the records yourself.
- For "Can Facility X perform Y?": Cross-reference their 'specialties' and 'equipment' columns. 
- For "Anomaly Detection": If a facility claims a complex specialty (e.g., Surgery) but has low capacity (<5 beds), flag it as a 'Suspicious Claim'.

--- CITATION & TRANSPARENCY MANDATE ---
- Every facility you mention MUST be accompanied by its [pk_unique_id] and [source_url].
- You must explain your "Plan" before answering: "I will first search the database for X, then verify the resource correlation for Y."

--- DATA CONTEXT ---
- Primary Table: workspace.default.ghana_health_facilities
- Key Columns: pk_unique_id, name, capacity, equipment, specialties, lat, lon.
"""

# Agent
tools = [query_hospital_database, get_facility_official_data, extract_medical_features,
         search_medical_notes, check_audit_discrepancy, calculate_distance_km ] # Add your other tools here
agent_executor = create_react_agent(llm, tools, prompt=system_prompt)

# TEST
def run_audit(question):
    print(f"🚀 MISSION: {question}")
    for output in agent_executor.stream({"messages": [("user", question)]}):
        for key, value in output.items():
            if key == "agent":
                print(f"🧠 PLAN: {value['messages'][0].content}")
            elif key == "tools":
                print(f"🛠️ TOOL OUTPUT: {value['messages'][0].content}")

run_audit("Where are the largest geographic 'cold spots' where a critical procedure is absent within 10 km")

/home/spark-5af66613-e521-4239-b813-26/.ipykernel/36127/command-7535413008589977-1045342753:162: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools, prompt=system_prompt)


🚀 MISSION: Where are the largest geographic 'cold spots' where a critical procedure is absent within 10 km
🧠 PLAN: 
🛠️ TOOL OUTPUT: 109.92
🧠 PLAN: 


{"ts": "2026-02-08 00:09:08.959", "level": "ERROR", "logger": "pyspark.sql.connect.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.INTERNAL\n\tdetails = \"[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `lat` cannot be resolved. Did you mean one of the following? [`area`, `logo`, `name`, `email`, `capacity`]. SQLSTATE: 42703; line 1 pos 151;\n'Project ['pk_unique_id, 'name, 'capacity, 'equipment, 'specialties]\n+- 'Filter (((NOT equipment#25131 LIKE %cardiology% AND ('lat > 5.0)) AND ('lat < 7.0)) AND (('lon > -2.0) AND ('lon < 0.0)))\n   +- SubqueryAlias workspace.default.ghana_health_facilities\n      +- Relation workspace.default.ghana_health_facilities[source_url#25125,name#25126,pk_unique_id#25127L,mongo_db#25128,specialties#25129,procedure#25130,equipment#25131,capability#25132,organization_type#251

🛠️ TOOL OUTPUT: SQL Syntax Error. Please provide a simpler WHERE condition. Error: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `lat` cannot be resolved. Did you mean one of the following? [`area`, `logo`, `name`, `email`, `capacity`]. SQLSTATE: 42703; line 1 pos 151;
'Project ['pk_unique_id, 'name, 'capacity, 'equipment, 'specialties]
+- 'Filter (((NOT equipment#25131 LIKE %cardiology% AND ('lat > 5.0)) AND ('lat < 7.0)) AND (('lon > -2.0) AND ('lon < 0.0)))
   +- SubqueryAlias workspace.default.ghana_health_facilities
      +- Relation workspace.default.ghana_health_facilities[source_url#25125,name#25126,pk_unique_id#25127L,mongo_db#25128,specialties#25129,procedure#25130,equipment#25131,capability#25132,organization_type#25133,content_table_id#25134,phone_numbers#25135,email#25136,websites#25137,officialwebsite#25138,yearestablished#25139,acceptsvolunteers#25140,facebooklink#25141,twitterlink#25142,linkedinlink#25143,instagramlink#25144,log

{"ts": "2026-02-08 00:09:58.618", "level": "ERROR", "logger": "pyspark.sql.connect.logging", "msg": "GRPC Error received", "context": {}, "exception": {"class": "_MultiThreadedRendezvous", "msg": "<_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.INTERNAL\n\tdetails = \"[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `lat` cannot be resolved. Did you mean one of the following? [`area`, `logo`, `name`, `email`, `capacity`]. SQLSTATE: 42703; line 1 pos 149;\n'Project ['pk_unique_id, 'name, 'capacity, 'equipment, 'specialties]\n+- 'Filter (((specialties#25561 LIKE %cardiology% AND ('lat > 5.0)) AND ('lat < 7.0)) AND (('lon > -2.0) AND ('lon < 0.0)))\n   +- SubqueryAlias workspace.default.ghana_health_facilities\n      +- Relation workspace.default.ghana_health_facilities[source_url#25557,name#25558,pk_unique_id#25559L,mongo_db#25560,specialties#25561,procedure#25562,equipment#25563,capability#25564,organization_type#25565

🧠 PLAN: 
🛠️ TOOL OUTPUT: SQL Syntax Error. Please provide a simpler WHERE condition. Error: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `lat` cannot be resolved. Did you mean one of the following? [`area`, `logo`, `name`, `email`, `capacity`]. SQLSTATE: 42703; line 1 pos 149;
'Project ['pk_unique_id, 'name, 'capacity, 'equipment, 'specialties]
+- 'Filter (((specialties#25561 LIKE %cardiology% AND ('lat > 5.0)) AND ('lat < 7.0)) AND (('lon > -2.0) AND ('lon < 0.0)))
   +- SubqueryAlias workspace.default.ghana_health_facilities
      +- Relation workspace.default.ghana_health_facilities[source_url#25557,name#25558,pk_unique_id#25559L,mongo_db#25560,specialties#25561,procedure#25562,equipment#25563,capability#25564,organization_type#25565,content_table_id#25566,phone_numbers#25567,email#25568,websites#25569,officialwebsite#25570,yearestablished#25571,acceptsvolunteers#25572,facebooklink#25573,twitterlink#25574,linkedinlink#25575,instagramlink#25

Trace(trace_id=tr-53d647a6d1c47cf0805deca8fb9749ab)